# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [15]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import time
from requests.exceptions import RequestException
import requests

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [16]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [17]:
import os
adv_api_key = os.getenv('ADV_KEY')
poly_api_key=os.getenv('POLY_KEY')

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



symbol = "AAPL"

# Get current stock price
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
r = requests.get(url)
data = r.json()
print("Overview:")
print(data)

url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}'
r = requests.get(url)
data = r.json()
print("Price:")
print(data)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
my_cols = ["Ticker", "Stock Price", "Marker Cap", "Num Shares To Buy"]
df = pd.DataFrame(columns=my_cols)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

# Get current stock price


df = pd.DataFrame(columns=my_cols)
for stock in stocks["Ticker"][:3]:
    # Get company overview for market cap
    overview_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={stock}&apikey={api_key}'
    overview_response = requests.get(overview_url)
    overview_data = overview_response.json()
    
    # Extract market cap
    market_cap = int(overview_data.get("MarketCapitalization", 0))
    
    # Get current stock price
    price_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={stock}&apikey={api_key}'
    price_response = requests.get(price_url)
    price_data = price_response.json()
    
    # Extract price
    price = float(price_data.get("Global Quote", {}).get("05. price", 0))
    
    # Calculate number of shares to buy (example logic)
    # Assuming you have some logic for this calculation
    num_shares = 0  # Replace with your actual calculation
    
    # Add to DataFrame
    df.loc[len(df)] = [stock, price, market_cap, num_shares]
    
    # Sleep to avoid hitting API rate limits
    time.sleep(1) 

print(df)

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [19]:
import datetime

# Get yesterday's date (or the most recent trading day)
# You may need to adjust this logic to get the most recent trading day
yesterday = datetime.date.today() - datetime.timedelta(days=1)
date_str = yesterday.strftime('%Y-%m-%d')

# Get grouped daily data for all stocks
url = f"https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date_str}?adjusted=true&apiKey={poly_api_key}"
response = requests.get(url)
data = response.json()

In [20]:

# Create DataFrame
my_cols = ["Ticker", "Stock Price", "Market Cap", "Num Shares To Buy"]
df = pd.DataFrame(columns=my_cols)

# Load S&P 500 tickers
sp500_tickers = set(stocks["Ticker"])  # Convert to a set for faster lookups

# Process results
if "results" in data:
    for result in data["results"]:
        ticker = result["T"]  # Ticker symbol
        
        # Only include S&P 500 stocks
        if ticker in sp500_tickers:
            # Use closing price as the stock price
            price = result["c"]
            
            # Add to DataFrame (without market cap for now)
            df.loc[len(df)] = [ticker, price, None, 0]

print(df)

Empty DataFrame
Columns: [Ticker, Stock Price, Market Cap, Num Shares To Buy]
Index: []


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

# Function to get market cap with retries
def get_market_cap(ticker, api_key, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            # Get ticker details including market cap
            ticker_url = f"https://api.polygon.io/v3/reference/tickers/{ticker}?apiKey={api_key}"
            ticker_response = requests.get(ticker_url)
            ticker_data = ticker_response.json()
            
            if ticker_response.status_code == 200 and "results" in ticker_data:
                market_cap = ticker_data["results"].get("market_cap", 0)
                return market_cap, None
            else:
                error_message = f"Error: Status code {ticker_response.status_code}, Message: {ticker_data.get('error', 'Unknown error')}"
                retries += 1
                if retries < max_retries:
                    # Wait 12 seconds to ensure we stay within 5 calls per minute
                    print(f"Error getting market cap for {ticker}. Retrying in 12 seconds...")
                    time.sleep(12)
                else:
                    return 0, error_message
        except Exception as e:
            retries += 1
            if retries < max_retries:
                # Wait 12 seconds to ensure we stay within 5 calls per minute
                print(f"Exception for {ticker}: {str(e)}. Retrying in 12 seconds...")
                time.sleep(12)
            else:
                return 0, str(e)
    
    return 0, "Max retries exceeded"

# Main code to update the DataFrame with market caps
for i, row in df.iterrows():
    ticker = row["Ticker"]
    
    # Get market cap with retry logic
    market_cap, error = get_market_cap(ticker, poly_api_key)
    
    if error:
        print(f"Failed to get market cap for {ticker}: {error}")
    else:
        df.at[i, "Market Cap"] = market_cap
        print(f"Updated {ticker} with market cap: ${market_cap}")
    
    # Sleep between requests to respect API rate limits (5 calls per minute = 12 seconds between calls)
    time.sleep(12)

print(df)

In [21]:
# First, read the S&P 500 stocks
stocks = pd.read_csv('sp_500_stocks.csv')

# Create DataFrame with the columns we need
my_cols = ["Ticker", "Stock Price", "Market Cap", "Num Shares To Buy"]
df = pd.DataFrame(columns=my_cols)

# Copy tickers from stocks DataFrame
df['Ticker'] = stocks['Ticker']

# Generate random stock prices between $10 and $1000
np.random.seed(42)  # For reproducibility
df['Stock Price'] = np.random.uniform(10, 1000, size=len(df)).round(2)

# Generate random market caps between $50B and $2T
df['Market Cap'] = np.random.randint(50_000_000_000, 2_000_000_000_000, size=len(df))

print(df.head())  # Print first 5 rows to verify

  Ticker  Stock Price     Market Cap Num Shares To Buy
0      A       380.79  1820098551692               NaN
1    AAL       951.21   243600806434               NaN
2    AAP       734.67  1136371312005               NaN
3   AAPL       602.67   399681802098               NaN
4   ABBV       164.46  1147712734916               NaN


In [22]:
porfolio_size = 1000

position_size = float(portfolio_size) / len(df.index)
for i in range(0, len(df['Ticker'])-1):
    df.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / df['Price'][i])
df

KeyError: 'Price'

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:

writer = pd.ExcelWriter("recom_trades.xlsx", engine="xlsxwriter")
df.to_excel(writer, "Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()